**Add env kernel**
1. `pip install --user ipykernel`
2. `python -m ipykernel install --user --name=env`

**Remove env kernel**
`jupyter kernelspec uninstall env`

In [2]:
from cs50 import SQL
import sqlite3 as lite
import pandas as pd
import numpy as np
import csv

**Process NSE & BSE Bhav copies**
1. Open both files
2. Remove trailing spaces using regex

In [3]:
nseBhavPath = f"bhavData/NSE-20210401.csv"
bseBhavPath = f"bhavData/BSE-20210401.csv"
nseBhav = pd.read_csv(nseBhavPath)
bseBhav = pd.read_csv(bseBhavPath)
bseBhav = bseBhav.replace({r"^\s*|\s*$":""}, regex=True)
nseBhav = nseBhav.replace({r"^\s*|\s*$":""}, regex=True)

**Slice NSEBHAV to have only ISIN, SYMBOL, and SERIES**
**Sanitize NSE GB Names to match with Zerodha**

In [4]:
nseBhav = nseBhav[["ISIN","SYMBOL", "SERIES", ]]
nseBhav["SYMBOL"] = np.where(nseBhav["SERIES"] == "GB", nseBhav["SYMBOL"] +"-" + nseBhav["SERIES"], nseBhav["SYMBOL"])
nseBhav.rename(columns={"ISIN": "isin", "SYMBOL": "nse_symbol", "SERIES":"nse_series"}, inplace=True)

In [5]:
bseBhav=bseBhav[["ISIN_CODE", "SC_CODE", "SC_NAME", "SC_GROUP", "SC_TYPE"]]
bseBhav.rename(columns = {"ISIN_CODE":"isin", "SC_CODE":"bse_sc_code", "SC_GROUP":"bse_sc_group", "SC_TYPE": "bse_sc_type"}, inplace=True)
bseBhav["bse_sc_code"] = bseBhav["bse_sc_code"].astype(str)

In [6]:
bhavList = pd.merge(nseBhav, bseBhav, on="isin", how="outer")

**Open scrip.db and delete previous version of data**

In [6]:
con = lite.connect("bhavData/bhavList.db")
cur = con.cursor()
try:
    cur.execute("DROP TABLE bhavList")
except Exception as e:
    print(e)

In [7]:
bhavList.to_sql("bhavList", con)
con.commit()
con.close()

In [13]:
import yfinance as yf

In [7]:
symbols = bhavList.to_dict("records")

In [33]:
for symbol in symbols[:5]:
    try:
        if symbol["isin"] == yf.Ticker(f"{symbol['nse_symbol']}.NS").isin:
            symbol["yf_nse_symbol"] = f"{symbol['nse_symbol']}.NS"
        else:
            symbol["yf_nse_symbol"] = "ISIN_NOT_FOUND"
    except Exception as e:
        symbol["yf_nse_symbol"] = "NOT_FOUND_ERROR"

DEBUG: Starting new HTTPS connection (1): finance.yahoo.com:443
DEBUG: https://finance.yahoo.com:443 "GET /quote/20MICRONS.NS HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): finance.yahoo.com:443
DEBUG: https://finance.yahoo.com:443 "GET /quote/20MICRONS.NS/holders HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): finance.yahoo.com:443
DEBUG: https://finance.yahoo.com:443 "GET /quote/20MICRONS.NS/financials HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): markets.businessinsider.com:443
DEBUG: https://markets.businessinsider.com:443 "GET /ajax/SearchController_Suggest?max_results=25&query=20%20MICRONS%20LTD HTTP/1.1" 200 1264
DEBUG: Starting new HTTPS connection (1): finance.yahoo.com:443
DEBUG: https://finance.yahoo.com:443 "GET /quote/21STCENMGM.NS HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): finance.yahoo.com:443
DEBUG: https://finance.yahoo.com:443 "GET /quote/21STCENMGM.NS/holders HTTP/1.1" 200 None
DEBUG: Starting new HTTPS conn

In [31]:
backup_symbols = symbols

In [36]:
out = pd.DataFrame(backup_symbols)
out.to_csv("yfrunbackukp.csv")

In [34]:
symbols[:5]

[{'isin': 'INE144J01027',
  'nse_symbol': '20MICRONS',
  'nse_series': 'EQ',
  'bse_sc_code': '533022',
  'SC_NAME': '20 MICRONS',
  'bse_sc_group': 'B',
  'bse_sc_type': 'Q',
  'yf_nse_symbol': '20MICRONS.NS'},
 {'isin': 'INE253B01015',
  'nse_symbol': '21STCENMGM',
  'nse_series': 'EQ',
  'bse_sc_code': '526921',
  'SC_NAME': '21ST CEN.MGM',
  'bse_sc_group': 'B',
  'bse_sc_type': 'Q',
  'yf_nse_symbol': 'ISIN_NOT_FOUND'},
 {'isin': 'INE748C01020',
  'nse_symbol': '3IINFOTECH',
  'nse_series': 'EQ',
  'bse_sc_code': '532628',
  'SC_NAME': '3I INFOTECH',
  'bse_sc_group': 'B',
  'bse_sc_type': 'Q',
  'yf_nse_symbol': 'ISIN_NOT_FOUND'},
 {'isin': 'INE470A01017',
  'nse_symbol': '3MINDIA',
  'nse_series': 'EQ',
  'bse_sc_code': '523395',
  'SC_NAME': '3M INDIA LTD',
  'bse_sc_group': 'A',
  'bse_sc_type': 'Q',
  'yf_nse_symbol': 'ISIN_NOT_FOUND'},
 {'isin': 'INE105C01023',
  'nse_symbol': '3PLAND',
  'nse_series': 'EQ',
  'bse_sc_code': '516092',
  'SC_NAME': '3PLAND',
  'bse_sc_group':

**Read this in more detail**
**https://towardsdatascience.com/the-unofficial-yahoo-finance-api-32dcf5d53df**

In [8]:
from yahooquery import Ticker

In [9]:
cesc = Ticker("cesc.ns")